<a href="https://colab.research.google.com/github/sourangshupal/Tensorflow2-Object-Detection-Tutorial/blob/master/object_detection_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial para Detecção de Objetos utilizando TensorFlow 2.x

Este tutorial é baseado nos trabalhos de [Hassan Rafiq](https://medium.com/swlh/image-object-detection-tensorflow-2-object-detection-api-af7244d4c34e) e [Dat Tran](https://towardsdatascience.com/how-to-train-your-own-object-detector-with-tensorflows-object-detector-api-bec72ecfe1d9).

## Checando & Instalando os pacotes

Primeiro passo é atualizar o gerenciador de pacotes `pip` para, em seguida, instalarmos os pacotes necessários.

In [1]:
!lsb_release -a

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 20.04.3 LTS
Release:	20.04
Codename:	focal


In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0


In [3]:
!pip install -U pip

In [ ]:
!wget https://developer.download.nvidia.com/compute/cuda/11.5.0/local_installers/cuda-repo-debian11-11-5-local_11.5.0-495.29.05-1_amd64.deb

In [ ]:
!dpkg -i cuda-repo-debian11-11-5-local_11.5.0-495.29.05-1_amd64.deb

In [ ]:
!apt-key add /var/cuda-repo-debian11-11-5-local/7fa2af80.pub
!sudo add-apt-repository contrib
!apt-get update
!apt-get -y install cuda-11-5

In [ ]:
# Donwloading libcudnn8 
!gdown --id  1a0Bfrt60W6NzP6LlEMGoOBg-26_vGlWa

In [ ]:
!dpkg -i libcudnn8_8.2.4.15-1+cuda11.4_amd64.deb

### Instalando tf slim

Vamos instalar o tf-slim, uyma API de alto nível para o tensorflow, utilizada para definição, treinamento e avaliação de modelos complexos. Mais informações em [https://github.com/google-research/tf-slim.](https://github.com/google-research/tf-slim)

In [ ]:
#!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim

### Verificando a Versão do Tensorflow

In [4]:
import tensorflow as tf
print(tf.__version__)

2.7.0


A saída esperada é Tensorflow==2.6.0

### Verificando se existe uma GPU disponível

In [5]:
tf.test.is_gpu_available() # True/False

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [6]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [7]:
!nvidia-smi

Mon Dec 13 17:55:19 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0  On |                  N/A |
| 45%   53C    P2    28W / 120W |    815MiB /  6075MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Vamos instalar o pacote `pycocotools`, uma API para ler e manipular as  notações do conjunto de imagens [COCO](https://cocodataset.org/).

In [ ]:
!pip install pycocotools

Mais informações em [https://medium.com/porto-seguro/precisa-de-imagens-para-treinar-sua-ia-use-a-api-do-coco-dataset-pycocotools-a87c7e21774b](https://medium.com/porto-seguro/precisa-de-imagens-para-treinar-sua-ia-use-a-api-do-coco-dataset-pycocotools-a87c7e21774b)

## Montando o Google Drive para armazenamento dos dados

Para montar sua pasta do Google Drive, você precisa logar na sua conta e fornecer um código de verificação, que será solicitado após a execução da célula a seguir.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Vamos criar um diretório no Google Drive, chamada `TFOD2`, a qual será nosso diretório de trabalho

In [ ]:
import os
os.makedirs('/content/drive/MyDrive/TFOD2', exist_ok=True)
os.chdir('/content/drive/MyDrive/TFOD2')

## Instalação API Object Detection 

Vamos baixar o repositório oficial do **Tensorflow**, disponível no github, mais especificiamente a pasta `tensorflow/models` o qual contém a API para detecção de objetos.  

In [ ]:
import os
import pathlib

os.chdir('/content/')

if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
    
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Agora, vamos compilar o protobufs e instalar o pacote para Object Detection.

In [ ]:
###%%bash
os.chdir('/content/models/research/')
!protoc object_detection/protos/*.proto --python_out=.

Para saber mais sobre o protobufs, visite [https://medium.com/trainingcenter/protobuf-uma-alternativa-ao-json-e-xml-a35c66edab4d](https://medium.com/trainingcenter/protobuf-uma-alternativa-ao-json-e-xml-a35c66edab4d).

Agora, vamos iniciar a instalação doa API do Tensorflow para Object Detection.

In [ ]:
!cp /content/models/research/object_detection/packages/tf2/setup.py /content/models/research
os.chdir('/content/models/research/')
!python -m pip install .

## Testando a instalação da API Object Detection

Executando a célula a seguir, será feito um teste que verifica se nossa instalação está correta.

In [ ]:
os.chdir('/content/models/research/')
!python object_detection/builders/model_builder_tf2_test.py

## Inferência: Detecção de Objetos utilizando modelo pré-treinado

Vamos realizar o processo de inferência para realizar a detecção de objetos utilizando o modelo pré-treinado no dataset [COCO](https://cocodataset.org/).

COCO é um dataset padrão para treinamento dos modelos para object detection, considerado um benchmark para comparação dos modelos.

Importando os pacotes necessários para inferência:

In [8]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

Importando o módulo de object detection:

In [ ]:
os.chdir('/content/drive/MyDrive/TFOD2/models/research/')

In [9]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Importando os Patches necessários.

In [10]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

## Preparando o Modelo

Vamos definir algumas funções necessárias para baixar o modelo e realizar o processo de inferência. Para isso, devemos escolher um modelo disponível em [detection model zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Nessa lista, podemos baixar modelos que forma exportados utilizando a ferramenta `export_inference_graph.py`, que permite realizar facilmente seu carregamento. Cada modelo possui uma determinada velocidade de execução e acurária, o que restringe sua aplicação.





### Definindo um Loader

In [11]:
# Verify: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

## Carregando um Label Map

Label Maps realizam o mapeamento de uma lista de índices para uma lista de categorias, portanto, quando nosso modelo predizer uma classe `5`, nós saberemos que se trata da classe `airplane`. Se preferir substituir por seu próprio Label Maps, basta criar uma função que retorne um dicionário, que mapeie números inteiros para o label das classes apropriadas.

Como o modelo que utilizaremos foi treinado no dataset COCO, vamos utilizar o Label Maps deste dataset.

In [ ]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/content/drive/MyDrive/TFOD2/models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

For the sake of simplicity we will test on 2 images:

## Carregando o modelo

Vamos definir um modelo, que será baixado e carregado na memória. Verifique um modelo dentre os disponíveis no  [Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md) do respositório oficial. 

Por exemplo, para baixar o modelo `mobilenet`, disponível no link 

[http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz](http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz)

basta indicar o nome do arquivo:
```
model_name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
```

Desta forma, você pode escolher e definir um modelo a ser utilizado.

In [ ]:
# os.chdir('/content/drive/MyDrive/TFOD2/')
# http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
model_name = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
detection_model = load_model(model_name)

Verificando a assinatura da entrada do modelo. Espera-se que receba um batch de imagens de 3 cores do tipo uint8:

In [ ]:
print(detection_model.signatures['serving_default'].inputs)
print(detection_model.signatures['serving_default'].output_dtypes)
print(detection_model.signatures['serving_default'].output_shapes)

Adicionando uma função wrapper para executar o processo de inferência utilizando o modelo escolhido sobre uma única imagem.

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Definindo uma função para exibição do resultado do processo de inferência.

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

### Teste de Inferência

Vamos definir algumas imagens disponíveis em uma pasta de teste da própria API.

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/drive/MyDrive/TFOD2/models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

Em seguida, vamos executar o processo de inferência sobre essas imagens:

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)


Para enviar uma imagem e executar o processo de inferência:

In [ ]:
from google.colab import files
from io import BytesIO
from PIL import Image

uploaded = files.upload()

for fn in uploaded.keys():
  show_inference(detection_model, fn)

## Criação e Preparação de um dataset customizado

Para criar o seu próprio dataset para detecção de objetos é necessário selecionar um conjunto de imagens do objeto que se deseja detectar e realizar a anotação das posições dos objetivos da imagem. Para isso, existe uma ferramenta chamada [LabelImg](https://github.com/tzutalin/labelImg) que facilita a etapa de anotação. 

Neste tutorial, utilizamos o formato XML para as anotações, conforme o padrão definido pelo dataset **Pascal VOC**. 

Para mais informações sobre o formado Pascal VOC, verifique [https://towardsdatascience.com/coco-data-format-for-object-detection-a4c5eaf518c5](https://towardsdatascience.com/coco-data-format-for-object-detection-a4c5eaf518c5).

Um tutorial para criação dos scripts  de exportação das anotações pode ser verificado em [https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md).

In [ ]:
# Source: https://medium.com/swlh/image-object-detection-tensorflow-2-object-detection-api-af7244d4c34e
import os
os.chdir('/content/')

Para este tutorial, utilizaremos o dataset [Raccoon Dataset](https://github.com/datitran/raccoon_dataset), que contém 200 imagens de guaxinins. No respositório deste dataset existe um conjunto de ferramentas que realizam a exportação das anotaçãos para o formato TFRecord, utilizado pelo Tensorflow.

Para mais informações sobre o formato TFRecord, verifique [https://blog.roboflow.com/create-tfrecord/](https://blog.roboflow.com/create-tfrecord/).


In [ ]:
!git clone https://github.com/freds0/capybara_dataset.git

In [ ]:
!ls capybara_dataset

### Covertendo XML para CSV

Dentre as ferramentas disponíveis no dataset [Raccoon Dataset](https://github.com/datitran/raccoon_dataset), utilizaremos as funções disponíveis em `generate_tfrecord.py`, especificamente as funções que convertem um arquivo XML para o formato CSV, que será posteriormente exportado para o formato TFRecord. 

In [ ]:
"""
source: https://github.com/datitran/raccoon_dataset/blob/master/generate_tfrecord.py

Usage:
  # From tensorflow/models/
  # Create train data:
  python generate_tfrecord.py --csv_input=data/train_labels.csv  --output_path=train.record
  # Create test data:
  python generate_tfrecord.py --csv_input=data/test_labels.csv  --output_path=test.record
"""
from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import pandas as pd
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util
from collections import namedtuple, OrderedDict

# TO-DO replace this with label map
def class_text_to_int(row_label):
    if row_label == 'capivara':
        return 1
    else:
        None

def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []

    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_text_to_int(row['class']))

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def generate_tfrecord(output_path, image_dir, csv_input):
    writer = tf.io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = pd.read_csv(csv_input)
    grouped = split(examples, 'filename')
    for group in grouped:
        tf_example = create_tf_example(group, path)
        writer.write(tf_example.SerializeToString())

    writer.close()
    output_path = os.path.join(os.getcwd(), output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))

### Exportando os arquivos de treino para o formato TFRecord

Nesta etapa, realizaremos a conversão dos arquivos CSV para o formato TFrecord, utilizando a `generate_tfrecord`definida na célula anterior. Os arquivos CSV encontram-se disponíveis no Raccoon Dataset.

In [ ]:
!ls /content/capybara_dataset/data/

Vamos exportar os arquivos de treino:

In [ ]:
# !python generate_tfrecord.py --csv_input=images/train_labels.csv --image_dir=images/train --output_path=train.record

##Run on training data
generate_tfrecord('/content/capybara_dataset/data/train.record', '/content/capybara_dataset/images/train/' , '/content/capybara_dataset/data/train_labels.csv')

Em seguida, vamos exportar os arquivos de teste:

In [ ]:
#!python generate_tfrecord.py --csv_input=images/test_labels.csv --image_dir=images/test --output_path=test.record
generate_tfrecord('/content/capybara_dataset/data/test.record', '/content/capybara_dataset/images/test' , '/content/capybara_dataset/data/test_labels.csv')


## Download e Treinamento do Modelo

Precisamos definir um modelo no qual será realizado *fine tuning* com o novo dataset. Escolha um modelo dentre os disponíveis no endereço [https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). 

Neste tutorial, vamos escolher o modelo **mobilenet_v2**.

In [ ]:
os.chdir('/content/')
!wget http://download.tensorflow.org/models/object_detection/classification/tf2/20200710/mobilenet_v2.tar.gz

Descompactando o modelo:

In [ ]:
import tarfile
#!tar -xvf mobilenet_v2.tar.gz
tar = tarfile.open("mobilenet_v2.tar.gz")
tar.extractall()
tar.close()

In [ ]:
os.listdir('mobilenet_v2/')

## Arquivos de Configuração

Para realizar o treinamento do modelo escolhido, devemos configurar dois arquivos:
- Label Maps
- Training Pipeline

### Label Maps

O arquivo Label Maps contém o dicionário das classes presentes no dataset. No Capybara Dataset esse arquivo possui o seguinte formato:

```
item {
  id: 1
  name: 'capivara'
}
```
Exemplos de arquivos Label Maps podem ser encontrados [neste link](https://github.com/tensorflow/models/tree/master/research/object_detection/data).
É importante notar que o label map deve sempre começar com id=1, conforme discutido [neste link] (https://github.com/tensorflow/models/issues/1696).

### Training Pipeline

O arquivo de Training Pipeline contém os parâmetros e hiperparâmetros para treinamento do modelo. 

Para mais informações sobre o pipeline de treinamento, verifique o conteúdo oficial do Tensorflow, disponível [neste link](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/configuring_jobs.md).

Para isso, utilizamos um arquivo de exemplo, que é específicio para o modelo escolhido. Os arquivos para cada modelo estão disponíveis [neste link](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

Para o modelo Mobilenet, utilizaremos o arquivo disponível no [repositório oficial do Tensorflow](https://github.com/tensorflow/models/blob/master/research/object_detection/samples/configs/ssd_mobilenet_v2_coco.config).

É necessário alterar o número de classes:

```
# Antes
num_classes: 90
# Depois
num_classes: 1	
```

```
# Antes
fine_tune_checkpoint: "PATH_TO_BE_CONFIGURED"
# Depois
fine_tune_checkpoint: "/content/mobilenet_v2/mobilenet_v2.ckpt-1"		##-- Edit( check downloaded model)
```

Em seguida, os paths dos arquivos de treinamento:
```
# Antes
train_input_reader: {
  label_map_path: "PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt"
  tf_record_input_reader {
    input_path: "PATH_TO_BE_CONFIGURED/mscoco_train.record-?????-of-00100"
  }  
}

# Depois
train_input_reader: {
  label_map_path: "/content/capybara_dataset/data/object-detection.pbtxt"
  tf_record_input_reader {
    input_path: "/content/capybara_dataset/data/train.record"
  }
}
```
E os paths dos arquivos de teste:
```
# Antes
eval_input_reader: {
  label_map_path: "PATH_TO_BE_CONFIGURED/mscoco_label_map.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "PATH_TO_BE_CONFIGURED/mscoco_val.record-?????-of-00010"
  }
}

# Depois
eval_input_reader: {
  label_map_path: "/content/capybara_dataset/data/object-detection.pbtxt"
  shuffle: false
  num_epochs: 1
  tf_record_input_reader {
    input_path: "/content/capybara_dataset/data/test.record" 
  }
}
```

Também é necessário definir um `batch_size` adequado à GPU disponível. Para uma Tesla T4, com aproximadamente 12G de memória dedicada, utilizamos o seguinte:
```
batch_size: 80
```


### Iniciando o Treinamento do Modelo

Por fim, precisamos apenas definir as ultimas configurações.

In [ ]:
os.chdir('/content/models/research/object_detection/')

In [ ]:
!python model_main_tf2.py \
    --model_dir='/content/mobilenet_v2_capybara_dataset/' \
    --pipeline_config_path='/content/mobilenet_v2_pipeline.config' \
    --checkpoint_every_n=1000 \
    --num_train_steps=10000

### Avaliando o Modelo Customizado

Após o treinamento, é possível avaliar os resultados.

In [ ]:
!python model_main_tf2.py \
    --model_dir='/content/drive/MyDrive/TFOD2/training_demo/mobilenet_v2/' \
    --pipeline_config_path='/content/mobilenet_v2_pipeline.config' \
    --checkpoint_dir='/content/mobilenet_v2_evaluation/'

Em seguida, utilize o Tensorboard para avaliar os resultados.

### Exportandos os checkpoints

Também é possíve exportar os checkpoints do modelo treinado, assim é possível enviar para uma outra plataforma a fim de realizar seu deploy.

In [ ]:
os.listdir('/content/')

In [ ]:
### Exporting the Infernece Graph
!python exporter_main_v2.py \
  --input_type image_tensor \
  --pipeline_config_path='/content/mobilenet_v2_pipeline.config' \
  --trained_checkpoint_dir='/content/mobilenet_v2_capybara_dataset/' \
  --output_directory='/content/mobilenet_v2_capybara_dataset/exported/'

## Inferência: Detecção de Objetos utilizando modelo treinado

In [ ]:
import pathlib 

def load_custom_model(model_name):
  model_file = model_name
  model_dir = pathlib.Path(model_file)/"saved_model"
  model = tf.saved_model.load(str(model_dir))

  return model

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/content/capybara_dataset/data/object-detection.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
model_name = '/content/mobilenet_v2_capybara_dataset/exported/'
detection_model = load_custom_model(model_name)

In [ ]:
print(detection_model.signatures['serving_default'].inputs)
print(detection_model.signatures['serving_default'].output_dtypes)
print(detection_model.signatures['serving_default'].output_shapes)

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

### Enviando um Arquivo para Inferência

In [ ]:
from google.colab import files
from io import BytesIO
from PIL import Image
import numpy as np

uploaded = files.upload()

for fn in uploaded.keys():
  show_inference(detection_model, fn)

### Inferência nos arquivos de exemplo

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/capybara_dataset/images/test')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpeg")))
TEST_IMAGE_PATHS

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)

In [ ]:
import shutil
import os
os.chdir('/content/mobilenet_v2_capybara_dataset')
output_zip_filename = 'mobilenet_v2_capybara_dataset'
shutil.make_archive(output_zip_filename, 'zip', 'exported')

In [ ]:
from google.colab import files

zip_file = '/content/mobilenet_v2_capybara_dataset/mobilenet_v2_capybara_dataset.zip'
files.download(zip_file)